In [27]:
import pandas as pd
input_file = "elevation_sweep_extra_2dB.csv"

In [28]:
import numpy
import matplotlib.pyplot as plt

# from "Security analysis of quantum key distribution with small block length and its application to quantum space communications

def shannon(Q):
    return -Q*numpy.log2(Q)-(1-Q)*numpy.log2(1-Q); 


def optKeyFrac(p, m, QBER, s = 6, f = 1.18):
    beta,xi, nu = p
    return 1-keyFraction(m, QBER, beta, xi, nu, s, f)

def keyFraction(m,QBER,beta,xi,nu, s = 6,f = 1.18): # m is the initial raw raw key length
    
    
    delta = QBER #QBER
    t = -numpy.log2(10**-(s+2)) #correctness
    r = f*shannon(delta) #leaked information
    epsQKD = 10**(-s)#2**-16 # qkd error 
    #m is the raw raw sifted key
    k = numpy.floor(beta*m)
    n = m-k # that's what's left after paramaeter estimation

    Gamma= 1/(m*(delta+xi)+ 1) +1/(m-m*(delta+xi)+ 1)
    PPElim = numpy.exp(-2*m*k*xi**2/(n+1)) + numpy.exp(-2*Gamma*((n*(nu-xi))**2-1))
    epe = numpy.sqrt(PPElim)
       
    tot = 4*(epsQKD - 2**-t - 2*epe)**2   
    
    l = numpy.log2(tot) - t +n*(1-shannon(delta+nu) - r)    
    epa = 0.5*numpy.sqrt(2**(-n*(1-shannon(delta+nu) - r )  + t + l))
   
    if epsQKD < 2**-t + 2*epe + epa:
        l =  -numpy.inf
        
#    if xi > nu:
#        l =  -numpy.inf
#        
#    if nu>0.5-delta:
#        l =  -numpy.inf
#        
#    if beta > 0.5:
#        l =  -numpy.inf
    return l


def optimize(n, qber, s = 10, f = 1.18, loops = 1, steps =101):
      
    par = [0,0,0]


    bounds= [[0,0.5],[0,0.5-qber],[0,0.5-qber]]


    lMax = -numpy.inf
    for loop in range(loops):
#        if bounds[0][0]<0:bounds[0][0]=0;
#        if bounds[0][0]<0:bounds[1][0]=0;
        arr0 = numpy.linspace(bounds[0][0], bounds[0][1] , steps)
        arr1 = numpy.linspace(bounds[1][0], bounds[1][1] , steps)
        
        for i in range(steps): 
            beta = arr0[i]

            for j in range(steps):
                nu = arr1[j]
#
#                if bounds[2][0] < 0:bounds[2][0] = 0
#                
                if bounds[2][1] > nu:
                    arr2 =  numpy.linspace(bounds[2][0], nu, steps)
                else:
                    arr2 =  numpy.linspace(bounds[2][0], bounds[2][1], steps)
                
                
                for q in range(steps):
                    xi = arr2[q]
                    
                    l = keyFraction(n, qber, beta,xi,nu,  s= s, f=f)
                    if l > lMax:
                        if beta > 0.5 or beta <= 0:
                            print('beta range error')
                            continue
                        if xi > nu or xi<=0:
                            print('xi<>nu comparison error')
                            continue
                        if nu > 0.5-qber or nu<=0:
                            print('nu error')
                            continue
                        lMax = l
                        par = [1*beta,1*xi,1*nu]
                        

        rng = 2
        bounds[0] = [par[0]-rng*(numpy.diff(arr0)[0]), par[0]+rng*(numpy.diff(arr0)[0])]
        bounds[1] = [par[2]-rng*(numpy.diff(arr1)[0]), par[2]+rng*(numpy.diff(arr1)[0])]
        bounds[2] = [par[1]-rng*(numpy.diff(arr2)[0]), par[1]+rng*(numpy.diff(arr2)[0])]

        steps = 11
        
    return lMax, par


In [29]:
df = pd.read_csv(input_file)
df

,Unnamed: 0,max_elev,QBER,sifted_key,total_coincidence,pass_duration
0,0,31,0.050973,3434.5,6842.5,57.5
1,1,32,0.050973,4688.5,9341.5,78.5
2,2,33,0.052774,5986.0,12019.0,94.5
3,3,34,0.053397,6890.5,13872.0,106.5
4,4,35,0.053695,7615.0,15351.0,116.5
5,5,36,0.054569,9048.5,18195.0,125.5
6,6,37,0.054862,9854.0,19806.0,132.5
7,7,38,0.055039,10570.0,21240.0,139.5
8,8,39,0.052142,11557.5,23214.0,144.5
9,9,40,0.051148,12331.5,24763.0,150.5


In [30]:
l_list = []

for index, row in df.iterrows(): 
    qber = row["QBER"]
    n = row["sifted_key"]
    l,par = optimize(n,qber)
    print (index, l, par)
    
    l_list.append(l)
    


0 -198.6275564487728 [0.5, 0.08477629759999998, 0.14368863999999998]
1 -118.77464433771878 [0.5, 0.07274237399999998, 0.12123728999999998]
2 -63.6634319667369 [0.47500000000000003, 0.0676205731195165, 0.10733424304685157]
3 2.98869185722819 [0.49, 0.060916612714382125, 0.09825260115222922]
4 53.15957392015025 [0.48, 0.059046094300600586, 0.09372395920730252]
5 155.28966503559855 [0.495, 0.052471769323788574, 0.0846318860061106]
6 188.49482680375812 [0.46, 0.05412874221878071, 0.08457615971684486]
7 190.07384938025317 [0.485, 0.05241641780337508, 0.08454260936028239]
8 522.2067717112621 [0.445, 0.051011005370010735, 0.07613582891046379]
9 664.0603228255804 [0.405, 0.05341343658468071, 0.07630490940668673]
10 764.8475271866844 [0.45, 0.048184549487072674, 0.07191723804040698]
11 954.8319786473755 [0.38, 0.05518810202582992, 0.07665014170254156]
12 1304.196950283825 [0.365, 0.053535526054546666, 0.07234530547911712]
13 1540.0036463153842 [0.38, 0.04890864829023391, 0.06792867818088043]
14

In [31]:
df['FiniteKey'] = l_list

In [32]:
df.to_csv("FiniteKey_2dB.csv")

In [33]:
df

,Unnamed: 0,max_elev,QBER,sifted_key,total_coincidence,pass_duration,FiniteKey
0,0,31,0.050973,3434.5,6842.5,57.5,-198.627556
1,1,32,0.050973,4688.5,9341.5,78.5,-118.774644
2,2,33,0.052774,5986.0,12019.0,94.5,-63.663432
3,3,34,0.053397,6890.5,13872.0,106.5,2.988692
4,4,35,0.053695,7615.0,15351.0,116.5,53.159574
5,5,36,0.054569,9048.5,18195.0,125.5,155.289665
6,6,37,0.054862,9854.0,19806.0,132.5,188.494827
7,7,38,0.055039,10570.0,21240.0,139.5,190.073849
8,8,39,0.052142,11557.5,23214.0,144.5,522.206772
9,9,40,0.051148,12331.5,24763.0,150.5,664.060323
